 # Optuna Merge

**Merge two dbs within Optuna**

# Initialization

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

## imports 

In [2]:
# Models
import os, sys
import math
import pickle
import itertools
import copy 
import joblib
import logging 
import types

from datetime import datetime, time
from collections.abc import Iterator
if './src' not in sys.path:
    print(f"insert ./src")
    sys.path.insert(0, './src')
print(sys.path)

import numpy as np
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')

import pandas as pd
pd.options.display.width = 170

import scipy.stats as sps 

import matplotlib.pyplot as plt

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)
from collections import defaultdict
from utils_cellpainting import *
from utils import display_gpu_info, display_gpu_device_info
# (initialize, init_dataloaders, init_environment, init_wandb, training_initializations, model_initializations, 


import warnings
warnings.filterwarnings('ignore')

insert ./src
['./src', '/home/kevin/WSL-shared/Cellpainting/cj-datasets', '/home/kevin/miniforge3/envs/cp311/lib/python311.zip', '/home/kevin/miniforge3/envs/cp311/lib/python3.11', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg']


/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [3]:
# display_gpu_info()

In [4]:
# import joblib
# from dask_cuda import LocalCUDACluster
# from sklearn.model_selection import GridSearchCV
import optuna



In [5]:
# os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train.ipynb"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [6]:
logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
FORMAT = '%(asctime)s - %(levelname)s: - %(message)s'
logging.basicConfig(level="INFO", format= FORMAT)
logging.getLogger("imported_module").setLevel(logging.CRITICAL)
# logging.info(f" 1/7- engine connected")
# logging.warning(f" 1/7- engine connected")
# logging.error(f" 1/7- engine connected")
# logging.critical(f" 1/7- engine connected")

# print(logging.NOTSET, logging.DEBUG,  logging.INFO, logging.WARN, logging.WARNING, logging.ERROR, logging.CRITICAL,  logging.FATAL)
# xgb.__version__

print()
for time_fmt in ['%x%X', '%X %x %Z', '%X.%f', '%D-%X.%f', '%Y-%m-%d %H:%M:%S.%f']:
    cmd_string = f"datetime.now().strftime('{time_fmt}')"
    print(f" {cmd_string:50s}  : {datetime.now().strftime(time_fmt)}")


 datetime.now().strftime('%x%X')                     : 04/22/2419:59:00
 datetime.now().strftime('%X %x %Z')                 : 19:59:00 04/22/24 
 datetime.now().strftime('%X.%f')                    : 19:59:00.479777
 datetime.now().strftime('%D-%X.%f')                 : 04/22/24-19:59:00.479789
 datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')     : 2024-04-22 19:59:00.479799


## Datasets

In [12]:
prefix = '' ### Target-2' , 'MOA'
db1  = f"./example.db"
db2  = f"./example_copy.db"

In [8]:
import sqlite3
import pandas as pd

In [9]:
def combine_optuna_dbs(db1_path, db2_path):
    """"
    This function combines two optuna databases into the db1.
    Both dbs should be from the same study.
    The ids of the db2 are updated to be unique.
    """
    
    # get all the tables names
    with sqlite3.connect(db1_path) as con:
        tables = con.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
        tables = [table[0] for table in tables]
    not_tables = ['studies', 'version_info', 'study_directions', 'alembic_version']
    tables = [table for table in tables if table not in not_tables]
    print(f"tbles are: {tables}")
    
    dfs1 = {}
    with sqlite3.connect(db1_path) as con:
        for table in tables:
            dfs1[table] = pd.read_sql_query(f"SELECT * FROM {table}", con)
    return dfs1
    table_ids = {'study_user_attributes'     : ['study_user_attribute_id'],
                 'study_system_attributes'   : ['study_system_attribute_id'],
                 'trials'                    : ['trial_id', 'number'],
                 'trial_user_attributes'     : ['trial_user_attribute_id'],
                 'trial_system_attributes'   : ['trial_system_attribute_id'],
                 'trial_params'              : ['param_id'],
                 'trial_values'              : ['trial_value_id'],
                 'trial_intermediate_values' : ['trial_intermediate_value_id'],
                 'trial_heartbeats'          : ['trial_heartbeat_id'],
    }
    
    max_ids = {}
    
    # # add max id if the table is not empty
    # for table in tables:
    #     if len(dfs1[table]) > 0:
    #         for id in table_ids[table]:
    #             max_ids[id] = dfs1[table].iloc[-1][id]
    #     else:
    #         for id in table_ids[table]:
    #             max_ids[id] = 0
    
    dfs2 = {}
    with sqlite3.connect(db2_path) as con:
        for table in tables:
            dfs2[table] = pd.read_sql_query(f"SELECT * FROM {table}", con)
    
    update_ids = {'study_user_attributes'     : ['study_user_attribute_id'],
                  'study_system_attributes'   : ['study_system_attribute_id'],
                  'trials'                    : ['trial_id', 'number'],
                  'trial_user_attributes'     : ['trial_user_attribute_id'  , 'trial_id'],
                  'trial_system_attributes'   : ['trial_system_attribute_id', 'trial_id'],
                  'trial_params'              : ['param_id', 'trial_id'],
                  'trial_values'              : ['trial_value_id', 'trial_id'],
                  'trial_intermediate_values' : ['trial_intermediate_value_id', 'trial_id'],
                  'trial_heartbeats'          : ['trial_heartbeat_id', 'trial_id'],
    }
    
    # # update the ids of the second db
    # for table in tables:
    #     if len(dfs2[table]) > 0:
    #         for id in update_ids[table]:
    #             dfs2[table][id] = dfs2[table][id] + max_ids[id]
    
    # # add the second db to the first db
    # with sqlite3.connect(db1_path) as con1:
    #     for table in tables:
    #         dfs2[table].to_sql(table, con1, if_exists='append', index=False)

In [13]:
tbls =  combine_optuna_dbs(db1, db2)

tbles are: ['study_user_attributes', 'study_system_attributes', 'trials', 'trial_user_attributes', 'trial_system_attributes', 'trial_params', 'trial_values', 'trial_intermediate_values', 'trial_heartbeats']


In [14]:
tbls.keys()

dict_keys(['study_user_attributes', 'study_system_attributes', 'trials', 'trial_user_attributes', 'trial_system_attributes', 'trial_params', 'trial_values', 'trial_intermediate_values', 'trial_heartbeats'])

In [24]:
tbls['study_user_attributes']
tbls['trial_user_attributes']
tbls['trial_intermediate_values']
tbls['trial_heartbeats']

tbls['study_system_attributes']
tbls['trial_system_attributes']
tbls['trials']
tbls['trial_params']
tbls['trial_values']

,study_user_attribute_id,study_id,key,value_json


,trial_user_attribute_id,trial_id,key,value_json


,trial_intermediate_value_id,trial_id,step,intermediate_value,intermediate_value_type


,trial_heartbeat_id,trial_id,heartbeat


,study_system_attribute_id,study_id,key,value_json
0,1,2,study:metric_names,"[""mse_score"", ""R2_score""]"
1,2,4,study:metric_names,"[""mse_score"", ""pearson_corr""]"
2,3,5,study:metric_names,"[""roc_auc"", ""logloss""]"


,trial_system_attribute_id,trial_id,key,value_json
0,1,17,nsga2:generation,0
1,2,18,nsga2:generation,0
2,3,19,nsga2:generation,0
3,4,20,nsga2:generation,0
4,5,21,nsga2:generation,0
...,...,...,...,...
99,100,116,nsga2:generation,0
100,101,117,nsga2:generation,0
101,102,118,nsga2:generation,0
102,103,119,nsga2:generation,0


,trial_id,number,study_id,state,datetime_start,datetime_complete
0,17,0,2,FAIL,2024-01-10 15:34:20.240464,2024-01-10 15:34:20.458984
1,18,1,2,COMPLETE,2024-01-10 15:51:14.898264,2024-01-10 16:00:22.971070
2,19,2,2,COMPLETE,2024-01-10 16:00:23.425742,2024-01-10 16:10:23.476871
3,20,3,2,COMPLETE,2024-01-10 16:10:23.958746,2024-01-10 16:20:46.199603
4,21,4,2,COMPLETE,2024-01-10 16:20:46.713100,2024-01-10 16:31:21.687379
...,...,...,...,...,...,...
99,116,25,5,COMPLETE,2024-02-12 01:55:54.321211,2024-02-12 02:12:44.511828
100,117,26,5,COMPLETE,2024-02-12 02:12:45.046738,2024-02-12 02:29:49.915460
101,118,27,5,COMPLETE,2024-02-12 02:29:50.425523,2024-02-12 03:59:12.655144
102,119,28,5,COMPLETE,2024-02-12 03:59:13.129237,2024-02-12 04:49:11.655533


,param_id,trial_id,param_name,param_value,distribution_json
0,49,17,learning_rate,0.021931,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
1,50,17,colsample_bytree,0.614273,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
2,51,17,colsample_bylevel,0.891675,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
3,52,17,colsample_bynode,0.598044,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
4,53,17,reg_lambda,0.677960,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
...,...,...,...,...,...
827,876,120,min_child_weight,8.575491,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
828,877,120,max_delta_step,7.047180,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
829,878,120,subsample,0.897760,"{""name"": ""FloatDistribution"", ""attributes"": {""..."
830,879,120,colsample_bylevel,0.860123,"{""name"": ""FloatDistribution"", ""attributes"": {""..."


,trial_value_id,trial_id,objective,value,value_type
0,15,18,0,0.023774,FINITE
1,16,18,1,-0.000156,FINITE
2,17,19,0,0.023621,FINITE
3,18,19,1,0.006289,FINITE
4,19,20,0,0.023774,FINITE
...,...,...,...,...,...
177,192,118,1,0.374718,FINITE
178,193,119,0,0.616641,FINITE
179,194,119,1,0.354772,FINITE
180,195,120,0,0.527256,FINITE
